In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis


import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Since I live in small town, the data of my town is not available on foursquare so here I am assuming singapore as my town. I did not design the framework of the report. The style of  this report is something that I copied from internet. THE CONTENT INSIDE IS MY ORIGINAL WORK

## REPORT CONTENT

1. Introduction Section: ⁃ Discussion of the business problem and the interested audience in this project.
2. Data Section: ⁃ Description of the data that will be used to solve the problem and the sources.
3. Methodology section: ⁃ Discussion and description of exploratory data analysis carried out, any inferential statistical testing performed, and if any machine learnings were used establishing the strategy and purposes.
4. Results section: ⁃ Discussion of the results.
5. Discussion section: ⁃ Elaboration and discussion on any observations noted and any recommendations suggested based on the results.
6. Conclusion section: ⁃ Report Conclusion.

# 1. Introduction

Discussion of the business problem and the audience who would be interested in this project.
Description of the Problem and Background

### Scenario:

I am residing in Downtown Singapore. I currently live within walking distance to Downtown Telok Ayer MRT metro station and I enjoy many ammenities and venues in the area, such as various international cousine restaurants, cafes, food shops and entertainment. I have been offered a great opportunity to study in Toronto, Canada. I am very excited and I want to use this opportunity to practice my learnings in Coursera in order to answer relevant questions arisen. The key question is : How can I find a convenient and enjoyable place similar to mine now in Singapore? Certainly, I can use available real estate apps and Google but the idea is to use and apply myself the learned tools during the course. In order to make a comparison and evaluation of the rental options in Manhattan NY, I must set some basis, therefore the apartment in Manhattan must meet the following demands:

- Apartment must be 2 or 3 bedrooms
- Desired location is near a metro station in the Toronto area and within 1.0 mile (1.6 km) radius
- Price of rent should not exceed $7,000 per month
- Top ammenities in the selected neighborhood shall be similar to current residence
- Desirable to have venues such as coffee shops, restaurants Asian Thai, wine stores, gym and food shops
- As a reference, I have included a map of venues near current residence in Singapore.


### Business Problem:
The challenge is to find a suitable apartment for rent in Toronto that complies with the demands on location, price and venues. The data required to resolve this challenge is described in the following section 2, below.

### Interested Audience
I believe this is a relevant challenge with valid questions for anyone moving to other large city in US, EU or Asia. The same methodology can be applied in accordance to demands as applicable. This case is also applicable for anyone interested in exploring starting or locating a new business in any city. Lastly, it can also serve as a good practical exercise to develop Data Science skills.

# 2. Data

### Description of the Data:

The following data is required to answer the issues of the problem:

- List of Boroughs and neighborhoods of Toronto with their geodata (latitud and longitud)
- List of Subway metro stations in Toronto with their address location
- List of apartments for rent in Toronto area with their addresses and price
- Preferably, a list of apartment for rent with additional information, such as price, address, area, # of beds, etc
- Venues for each Toronto neighborhood ( than can be clustered)
- Venues for subway metro stations, as needed

### How the data will be used to solve the problem

The data will be used as follows:

- Use Foursquare and geopy data to map top 10 venues for all Toronto neighborhoods and clustered in groups ( as per Course LAB)
- Use foursquare and geopy data to map the location of subway metro stations , separately and on top of the above clustered map in order to be able to identify the venues and ammenities near each metro station, or explore each subway location separately
- Use Foursquare and geopy data to map the location of rental places, in some form, linked to the subway locations.
- create a map that depicts, for instance, the average rental price per square ft, around a radious of 1.0 mile (1.6 km) around each subway station - or a similar metrics. I will be able to quickly point to the popups to know the relative price per subway area.
- Addresses from rental locations will be converted to geodata( lat, long) using Geopy-distance and Nominatim.

The procesing of these DATA will allow to answer the key questions to make a decision:

- what is the cost of rent (per square ft) around a mile radius from each subway metro station?
- what is the area of Toronto with best rental pricing that meets criteria established?
- What is the distance from Unievrsity and the tentative future home?
- What are the venues of the two best places to live? How the prices compare?
- How venues distribute among Toronto neighborhoods and around metro stations?
- Are there tradeoffs between size and price and location?
- Any other interesting statistical data findings of the real estate and overall data.

### Reference of venues around current residence in Singapore for comparison to Toronto place

In [7]:
address = 'Mccallum Street, Singapore'

geolocator = Nominatim()
location = geolocator.geocode(address)
neighborhood_latitude = location.latitude
neighborhood_longitude = location.longitude
print('The geograpical coordinate of Singapore home are {}, {}.'.format(latitude, longitude))

C:\Users\Nishant\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Singapore home are 1.2792423, 103.8481312.


In [11]:
CLIENT_ID = 'G2U1FJTHA2DHW55DHUT1JTWTSWV0GZHQ0HQ3XZAAHROJEDI3' # your Foursquare ID
CLIENT_SECRET = 'TTPKXE5D4XT2FWDKURQH2ROKP22WTFWXJDEG252235AMCGUD' # your Foursquare Secret
VERSION = '20180604'

In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=G2U1FJTHA2DHW55DHUT1JTWTSWV0GZHQ0HQ3XZAAHROJEDI3&client_secret=TTPKXE5D4XT2FWDKURQH2ROKP22WTFWXJDEG252235AMCGUD&v=20180604&ll=1.2792423,103.8481312&radius=500&limit=100'

In [14]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d73b509bcbf7a00367e65fe'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Financial District',
  'headerFullLocation': 'Financial District, Singapore',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 187,
  'suggestedBounds': {'ne': {'lat': 1.2837423045000045,
    'lng': 103.85262392222879},
   'sw': {'lat': 1.2747422954999954, 'lng': 103.84363847777121}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '51b49e49abd88dd0b4e7330f',
       'name': 'Napoleon Food & Wine Bar',
       'location': {'address': '206 Telok Ayer Street',
        'lat': 1.2799249387439204,
        'lng': 103.84733300574288,
        'labeledLatLngs': [{'la

In [15]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
venues = results['response']['groups'][0]['items']
    
SGnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues = SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(10)

,name,categories,lat,lng
0,Napoleon Food & Wine Bar,Wine Bar,1.279925,103.847333
1,Native,Cocktail Bar,1.280135,103.846844
2,Pepper Bowl,Asian Restaurant,1.279371,103.846710
3,Park Bench Deli,Deli / Bodega,1.279872,103.847287
4,Sofitel So Singapore,Hotel,1.280124,103.849867
5,Muchachos,Burrito Place,1.279072,103.847026
6,Freehouse,Beer Garden,1.281254,103.848513
7,Coffee Break,Coffee Shop,1.279529,103.846695
8,PS.Cafe,Café,1.280468,103.846264
9,Matt's | The Chocolate Shop,Dessert Shop,1.280462,103.846950


In [17]:
# create map of Singapore place  using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(SGnearby_venues['lat'], SGnearby_venues['lng'], SGnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_sg)  
    
map_sg